# Train and test YOLOv5 model

Install YOLOv5 and dependencies.

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

# Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/

# Optional: Unzip training data on google drive

In [ ]:
%cd /content/gdrive/MyDrive/
!unzip /content/gdrive/MyDrive/split_0038.zip

In [ ]:
%cp -anrv "/content/gdrive/MyDrive/split_0017" "/content/gdrive/MyDrive/YOLOv5_working_dir/split"

# Upload and unzip training data without using google drive

In [ ]:
%cd /content/yolov5
from google.colab import files

uploaded = files.upload()

Unzip data using shutil

In [ ]:
import shutil
shutil.unpack_archive('/content/yolov5/split_0043 (1).zip', '/content/yolov5/')

# Create yaml files for localization

In [ ]:
# create training .yaml file for YOLOv5:
# simply a copy of the yolo s (small), but with nc = X,
# because we only have X classes
# https://jooskorstanje.com/yolov5-training-a-custom-object-detection-model.html

%cd /content/yolov5
with open('object_train_yaml', 'w+') as file:
    file.write(
        """
        # parameters
        nc: 2  # number of classes

        depth_multiple: 0.33  # model depth multiple
        width_multiple: 0.50  # layer channel multiple

        anchors:
          - [10,13, 16,30, 33,23]  # P3/8
          - [30,61, 62,45, 59,119]  # P4/16
          - [116,90, 156,198, 373,326]  # P5/32

        # YOLOv5 v6.0 backbone
        backbone:
          # [from, number, module, args]
          [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
           [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
           [-1, 3, C3, [128]],
           [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
           [-1, 6, C3, [256]],
           [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
           [-1, 9, C3, [512]],
           [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
           [-1, 3, C3, [1024]],
           [-1, 1, SPPF, [1024, 5]],  # 9
          ]

        # YOLOv5 v6.0 head
        head:
          [[-1, 1, Conv, [512, 1, 1]],
           [-1, 1, nn.Upsample, [None, 2, 'nearest']],
           [[-1, 6], 1, Concat, [1]],  # cat backbone P4
           [-1, 3, C3, [512, False]],  # 13

           [-1, 1, Conv, [256, 1, 1]],
           [-1, 1, nn.Upsample, [None, 2, 'nearest']],
           [[-1, 4], 1, Concat, [1]],  # cat backbone P3
           [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

           [-1, 1, Conv, [256, 3, 2]],
           [[-1, 14], 1, Concat, [1]],  # cat head P4
           [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

           [-1, 1, Conv, [512, 3, 2]],
           [[-1, 10], 1, Concat, [1]],  # cat head P5
           [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

           [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
          ]
        """
    )

In [ ]:
# create training .yaml file for YOLOv5:
# simply a copy of the yolo m (medium), but with nc = X,
# because we only have X classes
# https://jooskorstanje.com/yolov5-training-a-custom-object-detection-model.html

%cd /content/yolov5
with open('object_train_yaml', 'w+') as file:
    file.write(
        """
        # Parameters
        nc: 1  # number of classes

        depth_multiple: 0.67  # model depth multiple
        width_multiple: 0.75  # layer channel multiple

        anchors:
          - [10,13, 16,30, 33,23]  # P3/8
          - [30,61, 62,45, 59,119]  # P4/16
          - [116,90, 156,198, 373,326]  # P5/32

        # YOLOv5 v6.0 backbone
        backbone:
          [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
           [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
           [-1, 3, C3, [128]],
           [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
           [-1, 6, C3, [256]],
           [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
           [-1, 9, C3, [512]],
           [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
           [-1, 3, C3, [1024]],
           [-1, 1, SPPF, [1024, 5]],  # 9
          ]

        # YOLOv5 v6.0 head
        head:
          [[-1, 1, Conv, [512, 1, 1]],
           [-1, 1, nn.Upsample, [None, 2, 'nearest']],
           [[-1, 6], 1, Concat, [1]],  # cat backbone P4
           [-1, 3, C3, [512, False]],  # 13

           [-1, 1, Conv, [256, 1, 1]],
           [-1, 1, nn.Upsample, [None, 2, 'nearest']],
           [[-1, 4], 1, Concat, [1]],  # cat backbone P3
           [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

           [-1, 1, Conv, [256, 3, 2]],
           [[-1, 14], 1, Concat, [1]],  # cat head P4
           [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

           [-1, 1, Conv, [512, 3, 2]],
           [[-1, 10], 1, Concat, [1]],  # cat head P5
           [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

           [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
          ]
        """
    )

In [ ]:
# directories to images (labels are inferred from directory structure)
%cd /content/yolov5
with open('object_data_yaml', 'w+') as file:
    file.write(
        """
        train: /content/yolov5/split_0043/images/train
        val: /content/yolov5/split_0043/images/valid
        test: /content/yolov5/split_0043/images/test

        nc: 1
        # names: ['tube_missing','tube_intact']
        # names: ['coating_degraded', 'coating_intact']
        # names: ['pylon']
        # names: ['mirror_good', 'mirror_major_damage', 'mirror_missing']
        # names: ['ball_joint']
        # names: ['Hot-Spot-Multi','Diode','Hot-Spot','Vegetation','No-Anomaly','Cell-Multi','Soiling','Cell','Offline-Module','Shadowing','Cracking','Diode-Multi']
        names: ['PV_module']
        """
    )

In [ ]:
# create custom hyperparameter file hyp.custom_1
# combines image augmentation settings of hyp.finetune with remaining settings from hyp.scratch

%cd /content/yolov5/data
with open('hyp.custom_1.yaml', 'w+') as file:
    file.write(
        """
        lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3) -> from hyp.scratch
        lrf: 0.2  # final OneCycleLR learning rate (lr0 * lrf) -> from hyp.scratch
        momentum: 0.937  # SGD momentum/Adam beta1 -> from hyp.scratch
        weight_decay: 0.0005  # optimizer weight decay 5e-4 -> from hyp.scratch
        warmup_epochs: 3.0  # warmup epochs (fractions ok) -> from hyp.scratch
        warmup_momentum: 0.8  # warmup initial momentum -> from hyp.scratch
        warmup_bias_lr: 0.1  # warmup initial bias lr -> from hyp.scratch
        box: 0.05  # box loss gain -> from hyp.scratch
        cls: 0.5  # cls loss gain -> from hyp.scratch
        cls_pw: 1.0  # cls BCELoss positive_weight -> from hyp.scratch
        obj: 1.0  # obj loss gain (scale with pixels) -> from hyp.scratch
        obj_pw: 1.0  # obj BCELoss positive_weight -> from hyp.scratch
        iou_t: 0.20  # IoU training threshold -> from hyp.scratch
        anchor_t: 4.0  # anchor-multiple threshold -> from hyp.scratch
        # anchors: 3  # anchors per output layer (0 to ignore) -> from hyp.scratch
        fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5) -> from hyp.scratch
        hsv_h: 0.0138  # image HSV-Hue augmentation (fraction) -> from hyp.finetune
        hsv_s: 0.664  # image HSV-Saturation augmentation (fraction) -> from hyp.finetune
        hsv_v: 0.464  # image HSV-Value augmentation (fraction) -> from hyp.finetune
        degrees: 0.373  # image rotation (+/- deg) -> from hyp.finetune
        translate: 0.245  # image translation (+/- fraction) -> from hyp.finetune
        scale: 0.898  # image scale (+/- gain) -> from hyp.finetune
        shear: 0.602  # image shear (+/- deg) -> from hyp.finetune
        perspective: 0.0  # image perspective (+/- fraction), range 0-0.001 -> from hyp.finetune
        flipud: 0.00856  # image flip up-down (probability) -> from hyp.finetune
        fliplr: 0.5  # image flip left-right (probability) -> from hyp.finetune
        mosaic: 1.0  # image mosaic (probability) -> from hyp.finetune
        mixup: 0.243  # image mixup (probability) -> from hyp.finetune
        copy_paste: 0.0 # added after 01.09.2021 because of error during training
        """
    )


In [ ]:
# create custom hyperparameter file hyp.custom_coating
# combines image augmentation settings of hyp.finetune with remaining settings from hyp.scratch
# no mosaic

%cd /content/yolov5/data
with open('hyp.custom_coating.yaml', 'w+') as file:
    file.write(
        """
        lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3) -> from hyp.scratch
        lrf: 0.2  # final OneCycleLR learning rate (lr0 * lrf) -> from hyp.scratch
        momentum: 0.937  # SGD momentum/Adam beta1 -> from hyp.scratch
        weight_decay: 0.0005  # optimizer weight decay 5e-4 -> from hyp.scratch
        warmup_epochs: 3.0  # warmup epochs (fractions ok) -> from hyp.scratch
        warmup_momentum: 0.8  # warmup initial momentum -> from hyp.scratch
        warmup_bias_lr: 0.1  # warmup initial bias lr -> from hyp.scratch
        box: 0.05  # box loss gain -> from hyp.scratch
        cls: 0.5  # cls loss gain -> from hyp.scratch
        cls_pw: 1.0  # cls BCELoss positive_weight -> from hyp.scratch
        obj: 1.0  # obj loss gain (scale with pixels) -> from hyp.scratch
        obj_pw: 1.0  # obj BCELoss positive_weight -> from hyp.scratch
        iou_t: 0.20  # IoU training threshold -> from hyp.scratch
        anchor_t: 4.0  # anchor-multiple threshold -> from hyp.scratch
        # anchors: 3  # anchors per output layer (0 to ignore) -> from hyp.scratch
        fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5) -> from hyp.scratch
        hsv_h: 0.0138  # image HSV-Hue augmentation (fraction) -> from hyp.finetune
        hsv_s: 0.664  # image HSV-Saturation augmentation (fraction) -> from hyp.finetune
        hsv_v: 0.464  # image HSV-Value augmentation (fraction) -> from hyp.finetune
        degrees: 0.373  # image rotation (+/- deg) -> from hyp.finetune
        translate: 0.245  # image translation (+/- fraction) -> from hyp.finetune
        scale: 0.898  # image scale (+/- gain) -> from hyp.finetune
        shear: 0.602  # image shear (+/- deg) -> from hyp.finetune
        perspective: 0.0  # image perspective (+/- fraction), range 0-0.001 -> from hyp.finetune
        flipud: 0.00856  # image flip up-down (probability) -> from hyp.finetune
        fliplr: 0.5  # image flip left-right (probability) -> from hyp.finetune
        mosaic: 0.0  # image mosaic (probability) -> from hyp.finetune
        mixup: 0.243  # image mixup (probability) -> from hyp.finetune
        """
    )


In [ ]:
# create custom hyperparameter file hyp.custom_coating
# combines image augmentation settings of hyp.finetune with remaining settings from hyp.scratch
# no image augmentation

%cd /content/yolov5/data
with open('hyp.custom_coating_2.yaml', 'w+') as file:
    file.write(
        """
        lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3) -> from hyp.scratch
        lrf: 0.2  # final OneCycleLR learning rate (lr0 * lrf) -> from hyp.scratch
        momentum: 0.937  # SGD momentum/Adam beta1 -> from hyp.scratch
        weight_decay: 0.0005  # optimizer weight decay 5e-4 -> from hyp.scratch
        warmup_epochs: 3.0  # warmup epochs (fractions ok) -> from hyp.scratch
        warmup_momentum: 0.8  # warmup initial momentum -> from hyp.scratch
        warmup_bias_lr: 0.1  # warmup initial bias lr -> from hyp.scratch
        box: 0.05  # box loss gain -> from hyp.scratch
        cls: 0.5  # cls loss gain -> from hyp.scratch
        cls_pw: 1.0  # cls BCELoss positive_weight -> from hyp.scratch
        obj: 1.0  # obj loss gain (scale with pixels) -> from hyp.scratch
        obj_pw: 1.0  # obj BCELoss positive_weight -> from hyp.scratch
        iou_t: 0.20  # IoU training threshold -> from hyp.scratch
        anchor_t: 4.0  # anchor-multiple threshold -> from hyp.scratch
        # anchors: 3  # anchors per output layer (0 to ignore) -> from hyp.scratch
        fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5) -> from hyp.scratch
        hsv_h: 0.0  # image HSV-Hue augmentation (fraction) -> from hyp.finetune
        hsv_s: 0.0  # image HSV-Saturation augmentation (fraction) -> from hyp.finetune
        hsv_v: 0.0  # image HSV-Value augmentation (fraction) -> from hyp.finetune
        degrees: 0.0  # image rotation (+/- deg) -> from hyp.finetune
        translate: 0.0  # image translation (+/- fraction) -> from hyp.finetune
        scale: 0.0  # image scale (+/- gain) -> from hyp.finetune
        shear: 0.0  # image shear (+/- deg) -> from hyp.finetune
        perspective: 0.0  # image perspective (+/- fraction), range 0-0.001 -> from hyp.finetune
        flipud: 0.0  # image flip up-down (probability) -> from hyp.finetune
        fliplr: 0.0  # image flip left-right (probability) -> from hyp.finetune
        mosaic: 0.0  # image mosaic (probability) -> from hyp.finetune
        mixup: 0.0  # image mixup (probability) -> from hyp.finetune
        """
    )


In [ ]:
# create custom hyperparameter file hyp.custom_coating
# combines image augmentation settings of hyp.finetune with remaining settings from hyp.scratch
# no HSV augmentation

%cd /content/yolov5/data
with open('hyp.custom_coating_3.yaml', 'w+') as file:
    file.write(
        """
        lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3) -> from hyp.scratch
        lrf: 0.2  # final OneCycleLR learning rate (lr0 * lrf) -> from hyp.scratch
        momentum: 0.937  # SGD momentum/Adam beta1 -> from hyp.scratch
        weight_decay: 0.0005  # optimizer weight decay 5e-4 -> from hyp.scratch
        warmup_epochs: 3.0  # warmup epochs (fractions ok) -> from hyp.scratch
        warmup_momentum: 0.8  # warmup initial momentum -> from hyp.scratch
        warmup_bias_lr: 0.1  # warmup initial bias lr -> from hyp.scratch
        box: 0.05  # box loss gain -> from hyp.scratch
        cls: 0.5  # cls loss gain -> from hyp.scratch
        cls_pw: 1.0  # cls BCELoss positive_weight -> from hyp.scratch
        obj: 1.0  # obj loss gain (scale with pixels) -> from hyp.scratch
        obj_pw: 1.0  # obj BCELoss positive_weight -> from hyp.scratch
        iou_t: 0.20  # IoU training threshold -> from hyp.scratch
        anchor_t: 4.0  # anchor-multiple threshold -> from hyp.scratch
        # anchors: 3  # anchors per output layer (0 to ignore) -> from hyp.scratch
        fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5) -> from hyp.scratch
        hsv_h: 0.0  # image HSV-Hue augmentation (fraction) -> from hyp.finetune
        hsv_s: 0.0  # image HSV-Saturation augmentation (fraction) -> from hyp.finetune
        hsv_v: 0.0  # image HSV-Value augmentation (fraction) -> from hyp.finetune
        degrees: 0.373  # image rotation (+/- deg) -> from hyp.finetune
        translate: 0.245  # image translation (+/- fraction) -> from hyp.finetune
        scale: 0.898  # image scale (+/- gain) -> from hyp.finetune
        shear: 0.602  # image shear (+/- deg) -> from hyp.finetune
        perspective: 0.0  # image perspective (+/- fraction), range 0-0.001 -> from hyp.finetune
        flipud: 0.00856  # image flip up-down (probability) -> from hyp.finetune
        fliplr: 0.5  # image flip left-right (probability) -> from hyp.finetune
        mosaic: 0.0  # image mosaic (probability) -> from hyp.finetune
        mixup: 0.243  # image mixup (probability) -> from hyp.finetune
        """
    )

In [ ]:
# create custom hyperparameter file hyp.custom_coating
# combines image augmentation settings of hyp.finetune with remaining settings from hyp.scratch
# no HSV augmentation

%cd /content/yolov5/data
with open('hyp.custom_coating_evolve.yaml', 'w+') as file:
    file.write(
        """
        lr0: 0.0107  # initial learning rate (SGD=1E-2, Adam=1E-3) -> from hyp.scratch
        lrf: 0.224  # final OneCycleLR learning rate (lr0 * lrf) -> from hyp.scratch
        momentum: 0.98  # SGD momentum/Adam beta1 -> from hyp.scratch
        weight_decay: 0.00098  # optimizer weight decay 5e-4 -> from hyp.scratch
        warmup_epochs: 2.51  # warmup epochs (fractions ok) -> from hyp.scratch
        warmup_momentum: 0.95  # warmup initial momentum -> from hyp.scratch
        warmup_bias_lr: 0.136  # warmup initial bias lr -> from hyp.scratch
        box: 0.0476  # box loss gain -> from hyp.scratch
        cls: 0.355  # cls loss gain -> from hyp.scratch
        cls_pw: 1.22  # cls BCELoss positive_weight -> from hyp.scratch
        obj: 0.298  # obj loss gain (scale with pixels) -> from hyp.scratch
        obj_pw: 0.858  # obj BCELoss positive_weight -> from hyp.scratch
        iou_t: 0.20  # IoU training threshold -> from hyp.scratch
        anchor_t: 2.54  # anchor-multiple threshold -> from hyp.scratch
        fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5) -> from hyp.scratch
        hsv_h: 0.015  # image HSV-Hue augmentation (fraction) -> from hyp.finetune
        hsv_s: 0.715  # image HSV-Saturation augmentation (fraction) -> from hyp.finetune
        hsv_v: 0.295  # image HSV-Value augmentation (fraction) -> from hyp.finetune
        degrees: 0.0  # image rotation (+/- deg) -> from hyp.finetune
        translate: 0.0536  # image translation (+/- fraction) -> from hyp.finetune
        scale: 0.827  # image scale (+/- gain) -> from hyp.finetune
        shear: 0.0  # image shear (+/- deg) -> from hyp.finetune
        perspective: 0.0  # image perspective (+/- fraction), range 0-0.001 -> from hyp.finetune
        flipud: 0.0  # image flip up-down (probability) -> from hyp.finetune
        fliplr: 0.367  # image flip left-right (probability) -> from hyp.finetune
        mosaic: 1.0  # image mosaic (probability) -> from hyp.finetune
        mixup: 0.0  # image mixup (probability) -> from hyp.finetune
        copy_paste: 0.0
        anchors: 2.15
        """
    )

In [ ]:
# create custom hyperparameter file hyp.custom_coating
# combines model settings of hyp.custom_coating_evolve with remaining settings from hyp.custom_coating

%cd /content/yolov5/data
with open('hyp.custom_coating_4.yaml', 'w+') as file:
    file.write(
        """
        lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3) -> from hyp.scratch
        lrf: 0.2  # final OneCycleLR learning rate (lr0 * lrf) -> from hyp.scratch
        momentum: 0.937  # SGD momentum/Adam beta1 -> from hyp.scratch
        weight_decay: 0.0005  # optimizer weight decay 5e-4 -> from hyp.scratch
        warmup_epochs: 3.0  # warmup epochs (fractions ok) -> from hyp.scratch
        warmup_momentum: 0.8  # warmup initial momentum -> from hyp.scratch
        warmup_bias_lr: 0.1  # warmup initial bias lr -> from hyp.scratch
        box: 0.05  # box loss gain -> from hyp.scratch
        cls: 0.5  # cls loss gain -> from hyp.scratch
        cls_pw: 1.0  # cls BCELoss positive_weight -> from hyp.scratch
        obj: 1.0  # obj loss gain (scale with pixels) -> from hyp.scratch
        obj_pw: 1.0  # obj BCELoss positive_weight -> from hyp.scratch
        iou_t: 0.20  # IoU training threshold -> from hyp.scratch
        anchor_t: 20.0  # anchor-multiple threshold -> from hyp.scratch
        fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5) -> from hyp.scratch
        hsv_h: 0.015  # image HSV-Hue augmentation (fraction) -> from hyp.finetune
        hsv_s: 0.715  # image HSV-Saturation augmentation (fraction) -> from hyp.finetune
        hsv_v: 0.295  # image HSV-Value augmentation (fraction) -> from hyp.finetune
        degrees: 0.0  # image rotation (+/- deg) -> from hyp.finetune
        translate: 0.0536  # image translation (+/- fraction) -> from hyp.finetune
        scale: 0.827  # image scale (+/- gain) -> from hyp.finetune
        shear: 0.0  # image shear (+/- deg) -> from hyp.finetune
        perspective: 0.0  # image perspective (+/- fraction), range 0-0.001 -> from hyp.finetune
        flipud: 0.0  # image flip up-down (probability) -> from hyp.finetune
        fliplr: 0.367  # image flip left-right (probability) -> from hyp.finetune
        mosaic: 1.0  # image mosaic (probability) -> from hyp.finetune
        mixup: 0.0  # image mixup (probability) -> from hyp.finetune
        copy_paste: 0.0
        #anchors: 2.15
        """
    )

In [ ]:
# create custom hyperparameter file hyp.custom_coating
# combines model settings of hyp.custom_coating_evolve with remaining settings from hyp.custom_coating

%cd /content/yolov5/data
with open('hyp.custom_glasstube.yaml', 'w+') as file:
    file.write(
        """
        lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3) -> from hyp.scratch
        lrf: 0.2  # final OneCycleLR learning rate (lr0 * lrf) -> from hyp.scratch
        momentum: 0.937  # SGD momentum/Adam beta1 -> from hyp.scratch
        weight_decay: 0.0005  # optimizer weight decay 5e-4 -> from hyp.scratch
        warmup_epochs: 3.0  # warmup epochs (fractions ok) -> from hyp.scratch
        warmup_momentum: 0.8  # warmup initial momentum -> from hyp.scratch
        warmup_bias_lr: 0.1  # warmup initial bias lr -> from hyp.scratch
        box: 0.05  # box loss gain -> from hyp.scratch
        cls: 0.5  # cls loss gain -> from hyp.scratch
        cls_pw: 1.0  # cls BCELoss positive_weight -> from hyp.scratch
        obj: 1.0  # obj loss gain (scale with pixels) -> from hyp.scratch
        obj_pw: 1.0  # obj BCELoss positive_weight -> from hyp.scratch
        iou_t: 0.20  # IoU training threshold -> from hyp.scratch
        anchor_t: 20.0  # anchor-multiple threshold -> from hyp.scratch
        fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5) -> from hyp.scratch
        hsv_h: 0.0  # image HSV-Hue augmentation (fraction) -> from hyp.finetune
        hsv_s: 0.0  # image HSV-Saturation augmentation (fraction) -> from hyp.finetune
        hsv_v: 0.0  # image HSV-Value augmentation (fraction) -> from hyp.finetune
        degrees: 0.0  # image rotation (+/- deg) -> from hyp.finetune
        translate: 0.0  # image translation (+/- fraction) -> from hyp.finetune
        scale: 0.0  # image scale (+/- gain) -> from hyp.finetune
        shear: 0.0  # image shear (+/- deg) -> from hyp.finetune
        perspective: 0.0  # image perspective (+/- fraction), range 0-0.001 -> from hyp.finetune
        flipud: 0.367  # image flip up-down (probability) -> from hyp.finetune
        fliplr: 0.367  # image flip left-right (probability) -> from hyp.finetune
        mosaic: 1.0  # image mosaic (probability) -> from hyp.finetune
        mixup: 0.0  # image mixup (probability) -> from hyp.finetune
        copy_paste: 0.0
        #anchors: 2.15
        """
    )

In [ ]:
# create custom hyperparameter file hyp.custom_coating
# combines model settings of hyp.custom_coating_evolve with remaining settings from hyp.custom_coating

%cd /content/yolov5/data
with open('hyp.custom_glasstube_no_mosaic.yaml', 'w+') as file:
    file.write(
        """
        lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3) -> from hyp.scratch
        lrf: 0.2  # final OneCycleLR learning rate (lr0 * lrf) -> from hyp.scratch
        momentum: 0.937  # SGD momentum/Adam beta1 -> from hyp.scratch
        weight_decay: 0.0005  # optimizer weight decay 5e-4 -> from hyp.scratch
        warmup_epochs: 3.0  # warmup epochs (fractions ok) -> from hyp.scratch
        warmup_momentum: 0.8  # warmup initial momentum -> from hyp.scratch
        warmup_bias_lr: 0.1  # warmup initial bias lr -> from hyp.scratch
        box: 0.05  # box loss gain -> from hyp.scratch
        cls: 0.5  # cls loss gain -> from hyp.scratch
        cls_pw: 1.0  # cls BCELoss positive_weight -> from hyp.scratch
        obj: 1.0  # obj loss gain (scale with pixels) -> from hyp.scratch
        obj_pw: 1.0  # obj BCELoss positive_weight -> from hyp.scratch
        iou_t: 0.20  # IoU training threshold -> from hyp.scratch
        anchor_t: 20.0  # anchor-multiple threshold -> from hyp.scratch
        fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5) -> from hyp.scratch
        hsv_h: 0.0  # image HSV-Hue augmentation (fraction) -> from hyp.finetune
        hsv_s: 0.0  # image HSV-Saturation augmentation (fraction) -> from hyp.finetune
        hsv_v: 0.0  # image HSV-Value augmentation (fraction) -> from hyp.finetune
        degrees: 0.0  # image rotation (+/- deg) -> from hyp.finetune
        translate: 0.0  # image translation (+/- fraction) -> from hyp.finetune
        scale: 0.0  # image scale (+/- gain) -> from hyp.finetune
        shear: 0.0  # image shear (+/- deg) -> from hyp.finetune
        perspective: 0.0  # image perspective (+/- fraction), range 0-0.001 -> from hyp.finetune
        flipud: 0.367  # image flip up-down (probability) -> from hyp.finetune
        fliplr: 0.367  # image flip left-right (probability) -> from hyp.finetune
        mosaic: 0.0  # image mosaic (probability) -> from hyp.finetune
        mixup: 0.0  # image mixup (probability) -> from hyp.finetune
        copy_paste: 0.0
        #anchors: 2.15
        """
    )

In [ ]:
# create custom hyperparameter file hyp.custom_coating
# combines model settings of hyp.custom_coating_evolve with remaining settings from hyp.custom_coating

%cd /content/yolov5/data
with open('hyp.custom_glasstube_2.yaml', 'w+') as file:
    file.write(
        """
        lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3) -> from hyp.scratch
        lrf: 0.2  # final OneCycleLR learning rate (lr0 * lrf) -> from hyp.scratch
        momentum: 0.937  # SGD momentum/Adam beta1 -> from hyp.scratch
        weight_decay: 0.0005  # optimizer weight decay 5e-4 -> from hyp.scratch
        warmup_epochs: 3.0  # warmup epochs (fractions ok) -> from hyp.scratch
        warmup_momentum: 0.8  # warmup initial momentum -> from hyp.scratch
        warmup_bias_lr: 0.1  # warmup initial bias lr -> from hyp.scratch
        box: 0.05  # box loss gain -> from hyp.scratch
        cls: 0.5  # cls loss gain -> from hyp.scratch
        cls_pw: 1.0  # cls BCELoss positive_weight -> from hyp.scratch
        obj: 1.0  # obj loss gain (scale with pixels) -> from hyp.scratch
        obj_pw: 1.0  # obj BCELoss positive_weight -> from hyp.scratch
        iou_t: 0.20  # IoU training threshold -> from hyp.scratch
        anchor_t: 20.0  # anchor-multiple threshold -> from hyp.scratch
        fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5) -> from hyp.scratch
        hsv_h: 0.0  # image HSV-Hue augmentation (fraction) -> from hyp.finetune
        hsv_s: 0.0  # image HSV-Saturation augmentation (fraction) -> from hyp.finetune
        hsv_v: 0.0  # image HSV-Value augmentation (fraction) -> from hyp.finetune
        degrees: 0.0  # image rotation (+/- deg) -> from hyp.finetune
        translate: 0.0  # image translation (+/- fraction) -> from hyp.finetune
        scale: 0.0  # image scale (+/- gain) -> from hyp.finetune
        shear: 0.0  # image shear (+/- deg) -> from hyp.finetune
        perspective: 0.0  # image perspective (+/- fraction), range 0-0.001 -> from hyp.finetune
        flipud: 0.0  # image flip up-down (probability) -> from hyp.finetune
        fliplr: 0.0  # image flip left-right (probability) -> from hyp.finetune
        mosaic: 0.0  # image mosaic (probability) -> from hyp.finetune
        mixup: 0.0  # image mixup (probability) -> from hyp.finetune
        copy_paste: 0.0
        #anchors: 2.15
        """
    )

In [ ]:
# create custom hyperparameter file hyp.custom_coating
# combines model settings of hyp.custom_coating_evolve with remaining settings from hyp.custom_coating

%cd /content/yolov5/data
with open('hyp.custom_coating_5.yaml', 'w+') as file:
    file.write(
        """
        lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3) -> from hyp.scratch
        lrf: 0.2  # final OneCycleLR learning rate (lr0 * lrf) -> from hyp.scratch
        momentum: 0.937  # SGD momentum/Adam beta1 -> from hyp.scratch
        weight_decay: 0.0005  # optimizer weight decay 5e-4 -> from hyp.scratch
        warmup_epochs: 3.0  # warmup epochs (fractions ok) -> from hyp.scratch
        warmup_momentum: 0.8  # warmup initial momentum -> from hyp.scratch
        warmup_bias_lr: 0.1  # warmup initial bias lr -> from hyp.scratch
        box: 0.05  # box loss gain -> from hyp.scratch
        cls: 0.5  # cls loss gain -> from hyp.scratch
        cls_pw: 1.0  # cls BCELoss positive_weight -> from hyp.scratch
        obj: 1.0  # obj loss gain (scale with pixels) -> from hyp.scratch
        obj_pw: 1.0  # obj BCELoss positive_weight -> from hyp.scratch
        iou_t: 0.20  # IoU training threshold -> from hyp.scratch
        anchor_t: 20.0  # anchor-multiple threshold -> from hyp.scratch
        fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5) -> from hyp.scratch
        hsv_h: 0.0  # image HSV-Hue augmentation (fraction) -> from hyp.finetune
        hsv_s: 0.715  # image HSV-Saturation augmentation (fraction) -> from hyp.finetune
        hsv_v: 0.295  # image HSV-Value augmentation (fraction) -> from hyp.finetune
        degrees: 0.0  # image rotation (+/- deg) -> from hyp.finetune
        translate: 0.0536  # image translation (+/- fraction) -> from hyp.finetune
        scale: 0.827  # image scale (+/- gain) -> from hyp.finetune
        shear: 0.0  # image shear (+/- deg) -> from hyp.finetune
        perspective: 0.0  # image perspective (+/- fraction), range 0-0.001 -> from hyp.finetune
        flipud: 0.0  # image flip up-down (probability) -> from hyp.finetune
        fliplr: 0.367  # image flip left-right (probability) -> from hyp.finetune
        mosaic: 1.0  # image mosaic (probability) -> from hyp.finetune
        mixup: 0.0  # image mixup (probability) -> from hyp.finetune
        copy_paste: 0.0
        #anchors: 2.15
        """
    )

# Train localization

Install weights and biases to track training progress in real time.

In [ ]:
# Weights & Biases (optional)
%pip install -q wandb
!wandb login  # use 'wandb disabled' or 'wandb enabled' to disable or enable

Train the model.

In [ ]:
%cd /content/yolov5
!wandb enabled
!python train.py --img 128 --batch 64 --epochs 300 --data 'object_data_yaml' --cfg 'object_train_yaml' --weights yolov5m.pt --name train_0076 --cache --hyp 'data/hyp.custom_glasstube.yaml'
%cp -anrv "/content/yolov5/runs/train/train_0076" "/content/gdrive/MyDrive/YOLOv5_working_dir/model/train"

In [ ]:
%cp -anrv "/content/yolov5/runs/train/train_00682" "/content/gdrive/MyDrive/YOLOv5_working_dir/model/train"

# Train classification

Download pretrained weights if required.

In [ ]:
%cd /content/yolov5
import utils
from utils.downloads import attempt_download

p5 = ['n', 's', 'm', 'l', 'x']  # P5 models
cls = [f'{x}-cls' for x in p5]  # classification models

for x in cls:
    attempt_download(f'weights/yolov5{x}.pt')

Train.

In [ ]:
%cd /content/yolov5
!wandb enabled
!python classify/train.py --img 64 --batch-size 64 --epochs 200 --data /content/yolov5/split_0038/images --model yolov5m-cls.pt --name train_0067 --cache
%cp -anrv "/content/yolov5/runs/train-cls/train_0067" "/content/gdrive/MyDrive/YOLOv5_working_dir/model/train"

In [ ]:
--pretrained weights/yolov5m-cls.pt

In [ ]:
!python export.py --weights runs/train-cls/train_0058/weights/best.pt --include onnx

Copy training result to google drive.

In [ ]:
%cp -anrv "/content/yolov5/runs/train-cls/train_0067" "/content/gdrive/MyDrive/YOLOv5_working_dir/model/train"

# Test localization
Test the model.

In [ ]:
%cd /content/yolov5
!python val.py --img 128 --data 'object_data_yaml' --weights /content/gdrive/MyDrive/YOLOv5_working_dir/model/train/train_0076/weights/last.pt --name test_0064 --task 'test' --save-txt --save-conf

Copy test results to google drive.

In [ ]:
%cp -anrv "/content/yolov5/runs/val/test_0064" "/content/gdrive/MyDrive/YOLOv5_working_dir/model/test"

# Test classification

In [ ]:
%cd /content/yolov5
!python classify/val.py --img 64 --data /content/gdrive/MyDrive/split_0038/images --weights /content/gdrive/MyDrive/YOLOv5_working_dir/model/train/train_0067/weights/last.pt --name test_0055

In [ ]:
%cp -anrv "/content/yolov5/runs/val-cls/test_0055" "/content/gdrive/MyDrive/YOLOv5_working_dir/model/test"

# Predict

In [ ]:
%cd /content/yolov5
!python detect.py --source '/content/gdrive/MyDrive/set_0007' --weights '/content/gdrive/MyDrive/YOLOv5_working_dir/model/train/train_0076/weights/last.pt' --img 640 --conf 0.9 --save-txt --name predict_0065

Copy prediction results to google drive.

In [ ]:
%cp -anrv "/content/yolov5/runs/detect/predict_0065" "/content/gdrive/MyDrive/YOLOv5_working_dir/model/predict/"

# Copy things around on google drive

In [ ]:
%cp -anrv "/content/gdrive/MyDrive/YOLOv5_working_dir/split/split_0003/images" "/content/gdrive/MyDrive/YOLOv5_working_dir/split/split_0005/images"

# Hyperparameter Evolution
Using genetic algorithm for 300 generations.

In [ ]:
%cd /content/yolov5
!wandb disabled
!python train.py --img 288 --batch 16 --epochs 10 --data 'object_data_yaml' --cfg 'object_train_yaml' --weights yolov5s.pt --name hyper_tune --cache --evolve

Without W&B

In [ ]:
%cd /content/yolov5
!python train.py --img 288 --batch 16 --epochs 10 --data 'object_data_yaml' --cfg 'object_train_yaml' --weights yolov5s.pt --name hyper_tune --cache --evolve